## Collaborative Recommendation System using VAECF (Cornac)

## 1. Objective  

The goal of this notebook is to build a **Collaborative Filtering Recommendation System** using the **Variational Autoencoder for Collaborative Filtering (VAECF)** model provided by the Cornac library.  

Unlike sequential or content-based recommenders, collaborative filtering relies purely on **user–item interaction patterns** (views and purchases). This allows the system to uncover latent user and item representations that drive recommendations.  

### Imports and Setups

In [ ]:
%pip install scipy
%pip install cornac
%pip install pandas
%pip install numpy
%pip install tensorflow
%pip install matplotlib
%pip install scikit-learn
%pip install torch

In [1]:
import warnings
warnings.simplefilter("ignore")

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print ("Number of GPUS present:",len(tf.config.list_physical_devices('GPU')))

In [ ]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import pandas as pd
from cornac.models.vaecf import VAECF
from cornac.eval_methods import RatioSplit
from cornac.metrics import Recall, NDCG
from cornac.data import Dataset
import torch


## 3. Data Loading  

In [7]:
data1 = pd.read_csv('/content/sample_data/processed_data.csv')
data2 = pd.read_csv('/content/sample_data/processed_data1.csv')

data = pd.concat([data1, data2], ignore_index=True)

In [34]:
data.head(20)

,user_id,product_id,event_type,brand,price,category_code,user,item,category_code_en,product_id_en,user_id_en,brand_en,price_norm,event_weight
0,561410373,1480163.0,1,acer,617.75,computers.desktop,1012839,2595,69,2595,1012839,4,0.239730,NaN
1,515179716,26900095.0,1,domyos,180.42,sport.trainer,190156,31070,125,31070,190156,394,0.069773,NaN
2,518000441,28712834.0,1,respect,79.02,apparel.shoes,299372,33425,12,33425,299372,1222,0.030367,NaN
3,539610377,2501799.0,1,elenberg,45.84,appliances.kitchen.oven,610826,5299,45,5299,610826,440,0.017472,NaN
4,513087375,52900016.0,1,stanley,11.39,accessories.bag,67931,41093,0,41093,67931,1369,0.004084,NaN
5,523500423,13200706.0,1,brw,258.18,furniture.bedroom.bed,423765,23471,105,23471,423765,240,0.099993,NaN
6,546652717,1004768.0,1,samsung,242.72,electronics.smartphone,710457,750,98,750,710457,1269,0.093985,NaN
7,518806907,1005263.0,1,oppo,772.19,electronics.smartphone,327255,1160,98,1160,327255,1096,0.299749,NaN
8,566307081,19200155.0,1,huter,52.23,construction.tools.saw,1057565,26777,83,26777,1057565,684,0.019956,NaN
9,518301968,10400757.0,1,wincars,42.70,kids.toys,305571,20499,119,20499,305571,1574,0.016252,NaN


## 4. Event Weighting  

To capture **interaction strength**, we assign weights:  

- `view` = **1.0**  
- `purchase` = **3.0**  

These are normalized to the range [0, 1]. Purchases therefore contribute more strongly to training.


In [ ]:


# Map weights before encoding
event_mapping = {
    "view": 1.0,
    "purchase": 3.0
}
data["event_weight"] = data["event_type"].map(event_mapping).fillna(0.0)
data["event_weight"] = data["event_weight"] / data["event_weight"].max()

# Now do label encoding


In [21]:
data['event_type'].head().unique()

array([1])

## 5. Encoding Users and Items  

We convert `user_id` and `product_id` into integer encodings for compatibility with Cornac.


In [ ]:
event_encode = LabelEncoder()
data['event_type'] = event_encode.fit_transform(data['event_type'])
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()
data['user'] = user_encoder.fit_transform(data['user_id'])
data['item'] = item_encoder.fit_transform(data['product_id'])


## 6. Train/Test Split  

We use Cornac’s `RatioSplit` to divide the dataset into **80% training** and **20% test**.  

- `rating_threshold=0.5` ensures only interactions above this weight are considered as positive feedback.


In [ ]:
interactions = list(zip(data['user'], data['item'], data['event_weight']))



# Split train/test
eval_method = RatioSplit(
    data=interactions,
    test_size=0.2,
    rating_threshold=0.5,  # this part is for implicit feedback
    exclude_unknowns=True,
    verbose=True
)



## 7. VAECF Model Training  

We train a **Variational Autoencoder for Collaborative Filtering (VAECF)** model with the following hyperparameters:  
Training runs on GPU if available.


In [ ]:
# VAE-CF model
model = VAECF(
    k=64,  # Latent factors
    autoencoder_structure=[200],
    n_epochs=10,
    batch_size=512,
    learning_rate=1e-4,
    use_gpu=True,  # set to false if cpu is used 
    verbose=True
)



# Train
history= model.fit(eval_method.train_set)
model.save("collab_recommender.keras")





rating_threshold = 0.5
exclude_unknowns = True
---
Training data:
Number of users = 1063162
Number of items = 39866
Number of ratings = 2284960
Max rating = nan
Min rating = nan
Global mean = nan
---
Test data:
Number of users = 1063162
Number of items = 39866
Number of ratings = 438248
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 1063162
Total items = 39866


  0%|          | 0/10 [00:00<?, ?it/s]

VAECF model is saved to collab_recommender.keras/VAECF/2025-08-25_14-19-26-363710.pkl


'collab_recommender.keras/VAECF/2025-08-25_14-19-26-363710.pkl'

### 8. Save the collaborative Recommender system

In [ ]:
vae.save("Collaborative_Recommender_Model")


### 9. Model loading 

In [ ]:

# Backup the original torch.load
original_torch_load = torch.load

# Define a patched version of torch.load that always forces CPU
def torch_load_cpu(*args, **kwargs):
    kwargs['map_location'] = torch.device('cpu')
    kwargs['weights_only'] = False  # Needed for PyTorch 2.6+
    return original_torch_load(*args, **kwargs)

# Patch torch.load
torch.load = torch_load_cpu

# Now load the Cornac model (this uses our patched torch.load)
model = VAECF.load("collab_recommender.keras/VAECF")

# Restore original torch.load after loading
torch.load = original_torch_load



### 10. EDA techniques before we make use of the model 

In [62]:
data = pd.read_csv("processed_data1.csv")

In [63]:
data.head()

,user_id,product_id,event_type,brand,price,category_code
0,555475417,1005115,view,apple,975.57,electronics.smartphone
1,532596496,2702282,view,lg,1846.74,appliances.kitchen.refrigerators
2,550048842,5100376,view,xiaomi,68.01,electronics.clocks
3,496957381,3601437,view,lg,380.85,appliances.kitchen.washer
4,524153511,1307401,view,asus,306.05,computers.notebook


### 11. Generating Recommendations  

We define a function `recommend_for_users()` that:  
- Scores all unseen items for a given user.  
- Returns **Top-N recommendations**.  
- Handles **cold-start users** by falling back to default items.


In [ ]:
def recommend_for_users(model, eval_method, user_ids, top_n=10, fallback_items=None):
    train_set = eval_method.train_set
    uid_map = train_set.uid_map
    iid_map = train_set.iid_map
    iid_map_inv = {v: k for k, v in iid_map.items()}

    results = {}

    for user_id_str in user_ids:
        print(f"\n User: {user_id_str}")

        if user_id_str not in uid_map:
            print("Cold-start user (not in training data).")

            if fallback_items:
                fallback_recs = fallback_items[:top_n]
                print(f"Fallback recommendations: {fallback_recs}")
                results[user_id_str] = [(item_id, None) for item_id in fallback_recs]
            else:
                print("No fallback available.")
                results[user_id_str] = []
            continue

        user_id = uid_map[user_id_str]
        n_items = train_set.num_items

        scores = [model.score(user_id, item_id) for item_id in range(n_items)]

        user_interactions = train_set.matrix[user_id]
        seen_items = set(user_interactions.indices)

        unseen_scores = [(item_id, score) for item_id, score in enumerate(scores) if item_id not in seen_items]
        top_items = sorted(unseen_scores, key=lambda x: x[1], reverse=True)[:top_n]
        rec_items = [(iid_map_inv[item_id], round(score, 4)) for item_id, score in top_items]

        for item_id, score in rec_items:
            print(f"🛍️ Item ID: {item_id} | Score: {score}")

        results[user_id_str] = rec_items

    return results








In [26]:
user_list = ['555475417','1005115']

In [81]:
model.device = torch.device('cpu')


## 12. Example Recommendation  

We now test the recommender on a sample user (`555475417`).


In [ ]:
recommendations = recommend_for_users(
    model=model, 
    eval_method=eval_method, 
    user_ids=['555475417'], 
    top_n=5,
   
)



👤 User: 555475417
🛍️  Item ID: 51700001 | Score: 0.03849999979138374
🛍️  Item ID: 3100105 | Score: 0.033900000154972076
🛍️  Item ID: 14700648 | Score: 0.02710000053048134
🛍️  Item ID: 11600064 | Score: 0.024800000712275505
🛍️  Item ID: 1005263 | Score: 0.018699999898672104


## 13. Overall Conclusion  

- The **collaborative VAECF model** successfully captured user–item interaction patterns.  
- Unlike content-based methods, it does not rely on product metadata, but instead learns from **implicit feedback signals** (views, purchases).  
- The system complements the **Sequential** and **Content-Based Recommenders**, creating a robust **hybrid recommendation pipeline**.  
